In [1]:
%matplotlib inline
import sys, os
sys.path.append('../..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [2]:
from sktime.base import load
#from sklearn.model_selection import GridSearchCV
#from sktime.classification.feature_based import TSFreshClassifier
from sktime.classification.deep_learning.cnn import CNNClassifier

In [3]:
period = 200
filepath = os.path.join('..','..','data','trend index','training set', f'trend_{period}day.hdf')
file = h5py.File(filepath, mode='r')

In [4]:
ohlc  = file['ohlc'][:]
trend = file['trend'][:]
# shuffling
idx = np.random.permutation(len(ohlc))
ohlc = ohlc[idx]
trend = trend[idx]

print(ohlc.shape)
print(trend.shape)


(1008, 4, 200)
(1008,)


In [5]:
file.close()

### 1. Train data & Test data 생성 (7:3)

In [6]:
idx = int(len(ohlc)*0.7)
ohlc_train = ohlc[:idx]
trend_train = trend[:idx]

ohlc_test = ohlc[idx:]
trend_test = trend[idx:]

print(ohlc_train.shape)
print(ohlc_test.shape)

(705, 4, 200)
(303, 4, 200)


### 2. 훈련 모델 생성 및 훈련

In [21]:
from sktime.transformations.series.impute import Imputer

In [23]:
transformer = Imputer(method="drift")

In [26]:
transformer.fit([ohlc_train[0]])

Imputer()

In [20]:
network = CNNClassifier(n_epochs=10,batch_size=4, verbose=True)
#network = TSFreshClassifier(verbose=True)
network.fit(ohlc_train, trend_train)

type: Data seen by CNNClassifier instance has missing values, but this CNNClassifier instance cannot handle missing values. Calls with missing values may result in error or unreliable results.

In [10]:
network.score(ohlc_test, trend_test)

76/76 [==============================] - 0s 696us/step


0.8316831683168316

In [11]:
network.predict_proba(ohlc_test)

76/76 [==============================] - 0s 700us/step


array([[8.10072676e-21, 3.81799042e-02, 9.61820126e-01],
       [2.39082538e-02, 9.76091743e-01, 1.24802668e-16],
       [9.99096870e-01, 9.03171953e-04, 1.81466582e-12],
       [9.48157310e-01, 5.18427081e-02, 2.48431827e-18],
       [9.99944746e-01, 5.52708116e-05, 7.94344688e-23],
       [1.34985685e-16, 9.95744348e-01, 4.25562914e-03],
       [9.99942601e-01, 5.74048245e-05, 6.98000211e-15],
       [7.32752056e-17, 2.53798004e-04, 9.99746263e-01],
       [2.28354323e-15, 9.99999940e-01, 4.94050703e-08],
       [3.04217011e-01, 6.95783019e-01, 9.04828546e-15],
       [4.95415106e-02, 9.50458467e-01, 1.01190689e-13],
       [1.39994216e-09, 9.94732022e-01, 5.26795862e-03],
       [3.93378332e-06, 9.99996066e-01, 7.50782357e-14],
       [2.86800777e-16, 1.09749519e-04, 9.99890268e-01],
       [1.95947081e-09, 3.32747586e-02, 9.66725230e-01],
       [7.92874235e-24, 3.81129881e-04, 9.99618888e-01],
       [7.08022764e-18, 1.55504349e-05, 9.99984443e-01],
       [2.04499418e-04, 9.99795

### 4. 검증

In [12]:
from tools.instruments import instruments
from tools.datatools import norm
from tools.display import highchart
quotes = instruments.quotes(fields='ohlc')
symbols = list(quotes.columns.levels[0])

In [13]:
%load_ext autoreload
%aimport tools.datatools
%aimport tools.display
%autoreload 2

In [18]:
symbol = np.random.choice(symbols)
quote = quotes[W'].dropna()
idx = 500#np.random.randint(0,len(quote)-51)
ohlc = quote.iloc[idx:idx+period]
data = norm(ohlc.values)
data = np.moveaxis(np.array([data]),1,2 )
print(network.predict_proba(data).round(2))
highchart([{'data':ohlc, 'type':'candlestick'}])

1/1 [==============================] - 0s 15ms/step
[[0. 1. 0.]]


### 3.모델을 파일로 저장

In [63]:
savepath = os.path.join('..','..','data','trend index','models', f'trend_{period}day')
network.save(savepath)

INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_50day\keras\assets


INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_50day\keras\assets


<zipfile.ZipFile filename='..\\..\\data\\trend index\\models\\trend_50day.zip' mode='r'>